In [21]:
import xgboost as xgb
import database
import numpy as np
import pandas as pd
import pickle
from tensorflow.keras.models import load_model
import pandas as pd
import json
from bs4 import BeautifulSoup
import regex as re
from tabulate import tabulate
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd
import psycopg2
from psycopg2 import sql


# Load the saved model


def update_dfs():
       

        driver = webdriver.Chrome()
        url = 'https://stats.wnba.com/players/boxscores-advanced/'
        driver.get(url)
        driver1 = webdriver.Chrome()
        url = 'https://stats.wnba.com/players/boxscores-scoring/'
        driver1.get(url)

        driver2 = webdriver.Chrome()
        url = 'https://stats.wnba.com/players/boxscores-traditional/'
        driver2.get(url)

        driver3 = webdriver.Chrome()
        url = 'https://stats.wnba.com/teams/boxscores-advanced/'
        driver3.get(url)


        driver4= webdriver.Chrome()
        url = 'https://stats.wnba.com/teams/boxscores-scoring/'
        driver4.get(url)


        driver5= webdriver.Chrome()
        url = 'https://stats.wnba.com/teams/boxscores-traditional/'
        driver5.get(url)




        input('press enter when all pages are set, advanced,scoring,traditional')


        html_content = driver.page_source


        soup = BeautifulSoup(html_content, 'html.parser')

        # Extract the table row data
        rows = soup.find_all('tr', {'data-ng-repeat': True})


        columns = [
        "Player", "Team", "Match Up", "Game Date", "Season", "W/L", "MIN", 
        "OffRtg", "DefRtg", "NetRtg", "AST%", "AST/TO", "AST Ratio", 
        "OREB%", "DREB%", "REB%", "TO Ratio", "eFG%", "TS%", "USG%", 
        "PACE", "PACE/40", "PIE"
        ]

        # Extracting the data
        data = []
        for row in rows:
                cols = row.find_all('td')
                cols = [col.get_text(strip=True) for col in cols]
                data.append(cols)

        # Creating the DataFrame
        df = pd.DataFrame(data, columns=columns)

        html_content = driver1.page_source


        soup = BeautifulSoup(html_content, 'html.parser')

        # Extract the table row data
        rows = soup.find_all('tr', {'data-ng-repeat': True})

        # Define the column names
        columns = [
        "Player", "Team", "Match Up", "Game Date", "Season", "W/L", "MIN", 
        "%FGA 2PT", "%FGA 3PT", "%PTS 2PT", "%PTS 2PT MR", "%PTS 3PT", 
        "%PTS FBPs", "%PTS FT", "%PTS OffTO", "%PTS PITP", "2FGM %AST", 
        "2FGM %UAST", "3FGM %AST", "3FGM %UAST", "FGM %AST", "FGM %UAST"
        ]
        data = []
        for row in rows:
                cols = row.find_all('td')
                cols = [col.get_text(strip=True) for col in cols]
                data.append(cols)

        # Creating the DataFrame
        tdf = pd.DataFrame(data, columns=columns)

        merged_df = pd.merge(df, tdf, on=['Player', 'Game Date'], suffixes=('', '_duplicate'))

        for col in merged_df.columns:
                if '_duplicate' in col:
                        original_col = col.replace('_duplicate', '')
                        merged_df[original_col] = merged_df[original_col].combine_first(merged_df[col])
                        merged_df.drop(columns=[col], inplace=True)

        html_content = driver2.page_source

        soup = BeautifulSoup(html_content, 'html.parser')

        # Extract the table row data
        rows = soup.find_all('tr', {'data-ng-repeat': True})

        columns = [
        "Player", "Team", "Match Up", "Game Date", "Season", "W/L", "MIN", 
        "PTS", "FGM", "FGA", "FG%", "3PM", "3PA", "3P%", "FTM", "FTA", "FT%", 
        "OREB", "DREB", "REB", "AST", "STL", "BLK", "TOV", "PF", "+/-"
        ]

        # Extracting the data
        data = []
        for row in rows:
                cols = row.find_all('td')
                cols = [col.get_text(strip=True) for col in cols]
                data.append(cols)

        # Creating the DataFrame
        df = pd.DataFrame(data, columns=columns)



        
        merged_df = pd.merge(merged_df, df, on = ['Player', 'Game Date'], suffixes=('', '_duplicate'))

        # Drop duplicate columns
        for col in merged_df.columns:
                if '_duplicate' in col:
                        original_col = col.replace('_duplicate', '')
                        merged_df[original_col] = merged_df[original_col].combine_first(merged_df[col])
                        merged_df.drop(columns=[col], inplace=True)
        player_df = merged_df.drop(columns=['Season']).dropna()


        ###########TEAM##################

        html_content = driver3.page_source


        soup = BeautifulSoup(html_content, 'html.parser')

        # Extract the table row data
        rows = soup.find_all('tr', {'data-ng-repeat': True})


        columns = [
                 "Team", "Match Up", "Game Date", "Season", "W/L", "MIN", 
        "OffRtg", "DefRtg", "NetRtg", "AST%", "AST/TO", "AST Ratio", 
        "OREB%", "DREB%", "REB%", "TOV%", "eFG%", "TS%",
        "PACE", "PACE/40", "PIE"
        ]

        # Extracting the data
        data = []
        for row in rows:
                cols = row.find_all('td')
                cols = [col.get_text(strip=True) for col in cols]
                data.append(cols)

        # Creating the DataFrame
        df = pd.DataFrame(data, columns=columns)

        html_content = driver4.page_source


        soup = BeautifulSoup(html_content, 'html.parser')

        # Extract the table row data
        rows = soup.find_all('tr', {'data-ng-repeat': True})

        # Define the column names
        columns = [
         "Team", "Match Up", "Game Date", "Season", "W/L", "MIN", 
        "%FGA 2PT", "%FGA 3PT", "%PTS 2PT", "%PTS 2PT MR", "%PTS 3PT", 
        "%PTS FBPs", "%PTS FT", "%PTS OffTO", "%PTS PITP", "2FGM %AST", 
        "2FGM %UAST", "3FGM %AST", "3FGM %UAST", "FGM %AST", "FGM %UAST"
        ]
        data = []
        for row in rows:
                cols = row.find_all('td')
                cols = [col.get_text(strip=True) for col in cols]
                data.append(cols)

        # Creating the DataFrame
        tdf = pd.DataFrame(data, columns=columns)

        merged_df = pd.merge(df, tdf, on=['Team', 'Game Date'], suffixes=('', '_duplicate'))

        for col in merged_df.columns:
                if '_duplicate' in col:
                        original_col = col.replace('_duplicate', '')
                        merged_df[original_col] = merged_df[original_col].combine_first(merged_df[col])
                        merged_df.drop(columns=[col], inplace=True)
        html_content = driver5.page_source

        soup = BeautifulSoup(html_content, 'html.parser')

        # Extract the table row data
        rows = soup.find_all('tr', {'data-ng-repeat': True})

        columns = [
        "Team", "Match Up", "Game Date", "Season", "W/L", "MIN", 
        "PTS", "FGM", "FGA", "FG%", "3PM", "3PA", "3P%", "FTM", "FTA", "FT%", 
        "OREB", "DREB", "REB", "AST", "STL", "BLK", "TOV", "PF", "+/-"
        ]

        # Extracting the data
        data = []
        for row in rows:
                cols = row.find_all('td')
                cols = [col.get_text(strip=True) for col in cols]
                data.append(cols)

        # Creating the DataFrame
        df = pd.DataFrame(data, columns=columns)



        
        merged_df = pd.merge(merged_df, df, on = ['Team', 'Game Date'], suffixes=('', '_duplicate'))

        # Drop duplicate columns
        for col in merged_df.columns:
                if '_duplicate' in col:
                        original_col = col.replace('_duplicate', '')
                        merged_df[original_col] = merged_df[original_col].combine_first(merged_df[col])
                        merged_df.drop(columns=[col], inplace=True)

                        
        return player_df ,merged_df.drop(columns=['Season']).dropna() 
player_df,teams_df= update_dfs()

# db = database.Database()

# stats,team_stats = db.get_updates(df, '2024-09-05')
# db.rebinsert_stats(stats,team_stats)
# db.insert_stats(stats,team_stats)


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=128.0.6613.120)
Stacktrace:
	GetHandleVerifier [0x00007FF7356A9412+29090]
	(No symbol) [0x00007FF73561E239]
	(No symbol) [0x00007FF7354DB1DA]
	(No symbol) [0x00007FF7354AFAF5]
	(No symbol) [0x00007FF73555E2C7]
	(No symbol) [0x00007FF735575EB1]
	(No symbol) [0x00007FF735556493]
	(No symbol) [0x00007FF7355209D1]
	(No symbol) [0x00007FF735521B31]
	GetHandleVerifier [0x00007FF7359C871D+3302573]
	GetHandleVerifier [0x00007FF735A14243+3612627]
	GetHandleVerifier [0x00007FF735A0A417+3572135]
	GetHandleVerifier [0x00007FF735765EB6+801862]
	(No symbol) [0x00007FF73562945F]
	(No symbol) [0x00007FF735624FB4]
	(No symbol) [0x00007FF735625140]
	(No symbol) [0x00007FF73561461F]
	BaseThreadInitThunk [0x00007FFE0908257D+29]
	RtlUserThreadStart [0x00007FFE0936AF28+40]


In [170]:
p = teams_df
c = player_df

In [203]:
def get_updates(self, df,team_df, last_date):
        conn = self.conn
        
        df = self.type_maker(df)
        self.opp(df)
        df['Game Date'] = pd.to_datetime(df['Game Date'], format='%m/%d/%Y')
        df['PLAYER'] = df['Player'].str.replace("'",'')
        all_df_att = df.sort_values('Game Date')
        all_df_att['Date'] = all_df_att['Game Date']
        all_df_att.drop(columns=['Game Date','Player'],inplace =True)
        all_df_att = all_df_att[all_df_att['Date']> last_date].dropna()

        player_df = all_df_att



        df = self.type_maker(team_df)
        self.opp(df)
        df['Game Date'] = pd.to_datetime(df['Game Date'], format='%m/%d/%Y')
        all_df_att = df.sort_values('Game Date')
        all_df_att['Date'] = all_df_att['Game Date']
        all_df_att.drop(columns=['Game Date'],inplace =True)
        all_df_att = all_df_att[all_df_att['Date']> last_date].dropna()


        team_stat_df = all_df_att

        query = f"SELECT * FROM pos_player;"

                # Execute the query and load the data into a DataFrame
        zdf = pd.read_sql_query(query, conn)

        zdf['PLAYER'] = zdf['player'].str.replace("'",'')
        zdf.drop(columns=['player'],inplace=True)


        all_df_att = pd.merge(player_df,zdf, how='left', on=['PLAYER'])


        pos_team_df = all_df_att.groupby(['Team', 'opp', 'Date','pos'])[['PTS', 'REB', 'OREB', 'DREB', 'FGA', 'AST']].sum().reset_index()



        return player_df.sort_values.sort_values('Date'), team_stat_df.sort_values('Date'), pos_team_df.sort_values('Date')

In [208]:
pos_team_df.sort_values('Date')

,Team,opp,Date,pos,PTS,REB,OREB,DREB,FGA,AST
120,NYL,CON,2024-08-24,0.0,42.0,36.0,10.0,26.0,39.0,10.0
38,CON,NYL,2024-08-24,0.0,27.0,13.0,4.0,9.0,23.0,8.0
39,CON,NYL,2024-08-24,1.0,45.0,24.0,4.0,20.0,49.0,12.0
121,NYL,CON,2024-08-24,1.0,22.0,8.0,1.0,7.0,26.0,9.0
74,IND,MIN,2024-08-24,0.0,24.0,29.0,12.0,17.0,33.0,4.0
...,...,...,...,...,...,...,...,...,...,...
107,MIN,ATL,2024-09-10,1.0,29.0,10.0,1.0,9.0,24.0,11.0
123,NYL,DAL,2024-09-10,1.0,34.0,12.0,1.0,11.0,28.0,24.0
122,NYL,DAL,2024-09-10,0.0,71.0,18.0,4.0,14.0,42.0,9.0
34,CON,LAS,2024-09-10,0.0,34.0,22.0,3.0,19.0,27.0,14.0


In [173]:
def opp(df):
        df['opp'] = df['Match Up'].apply(lambda x: x.split()[-1])
        df['home'] = df['Match Up'].apply(lambda x: 1 if '@' not in x.split() else 0)
        return 1

In [174]:
def type_maker(df):
        columns = [
             'OffRtg', 'DefRtg', 'NetRtg', 'AST%', 'AST/TO', 'AST Ratio',
                'OREB%', 'DREB%', 'REB%', 'TO Ratio', 'eFG%', 'TS%', 'USG%', 'PACE', 'PACE/40', 'PIE', '%FGA 2PT',
                '%FGA 3PT', '%PTS 2PT', '%PTS 2PT MR', '%PTS 3PT', '%PTS FBPs', '%PTS FT', '%PTS OffTO', '%PTS PITP',
                '2FGM %AST', '2FGM %UAST', '3FGM %AST', '3FGM %UAST', 'FGM %AST', 'FGM %UAST', 'PTS', 'FGM', 'FGA',
                'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV','TOV%',
                'PF', '+/-'
            ]
        
        for col in columns:
            if col in df.columns:  # Check if the column exists in the DataFrame
                if df[col].dtype == 'object':
                    df[col] = df[col].str.replace(',', '')
                try:
                    df[col] = df[col].astype(float)
                except ValueError:
                    print(f"Warning: Column '{col}' contains non-numeric values that could not be converted.")
    
        return df

In [195]:
conn = psycopg2.connect(
                dbname="wnbaall",
                user="postgres",
                password="5111",
                host="localhost",
                port="5432"
                )
 
                # Execute the query and load the data into a DataFrame


player_df, team_stat_df, pos_team_df = get_updates(conn,player_df,teams_df,'2024-08-23')

In [176]:
team_stat_df.columns = [col.upper() for col in team_stat_df.columns]

In [200]:
def rebinsert_stats(self,df,team_df,pos_df, close = False):
        conn = self.conn
        with conn.cursor() as cur:

            print(df['PTS'])

            for index, row in df.iterrows():

                player = row['PLAYER']


                # Retrieve the current values from the table
                cur.execute(
                    sql.SQL("""
                        SELECT fga, fgm, threepa, threepm, fta, ftm, pts, reb, oreb, dreb, ast, tov, pf, stl, blk, pace, pace_per_40, pie, 
                            percentagefga_twopt, percentagefga_threept, percentagepts_twopt, percentagepts_twopt_mr, 
                            percentagepts_threept, percentagepts_fbps, percentagepts_ft, percentagepts_offto, percentagepts_pitp, 
                            twofgm_percentageast, twofgm_percentageuast, threefgm_percentageast, threefgm_percentageuast, 
                            fgm_percentageast, fgm_percentageuast, usgpercentage, rebpercentage, games_played 
                        FROM stat_avg_season 
                        WHERE player = %s
                    """),
                    [player]
                )

                result = cur.fetchone()
                if result:
                    (current_fga, current_fgm, current_threepa, current_threepm, current_fta, current_ftm, current_pts, current_reb, 
                    current_oreb, current_dreb, current_ast, current_tov, current_pf, current_stl, current_blk, current_pace, 
                    current_pace_per_40, current_pie, current_percentagefga_twopt, current_percentagefga_threept, 
                    current_percentagepts_twopt, current_percentagepts_twopt_mr, current_percentagepts_threept, 
                    current_percentagepts_fbps, current_percentagepts_ft, current_percentagepts_offto, current_percentagepts_pitp, 
                    current_twofgm_percentageast, current_twofgm_percentageuast, current_threefgm_percentageast, 
                    current_threefgm_percentageuast, current_fgm_percentageast, current_fgm_percentageuast, current_usgpercentage, 
                    current_rebpercentage, games_played) = result

                    # Calculate new averages
                    games_played += 1
                    updated_row = [
                        (current_fga * (games_played - 1) + row['FGA']) / games_played,
                        (current_fgm * (games_played - 1) + row['FGM']) / games_played,
                        (current_threepa * (games_played - 1) + row['3PA']) / games_played,
                        (current_threepm * (games_played - 1) + row['3PM']) / games_played,
                        (current_fta * (games_played - 1) + row['FTA']) / games_played,
                        (current_ftm * (games_played - 1) + row['FTM']) / games_played,
                        (current_pts * (games_played - 1) + row['PTS']) / games_played,
                        (current_reb * (games_played - 1) + row['REB']) / games_played,
                        (current_oreb * (games_played - 1) + row['OREB']) / games_played,
                        (current_dreb * (games_played - 1) + row['DREB']) / games_played,
                        (current_ast * (games_played - 1) + row['AST']) / games_played,
                        (current_tov * (games_played - 1) + row['TOV']) / games_played,
                        (current_pf * (games_played - 1) + row['PF']) / games_played,
                        (current_stl * (games_played - 1) + row['STL']) / games_played,
                        (current_blk * (games_played - 1) + row['BLK']) / games_played,
                        (current_pace * (games_played - 1) + row['PACE']) / games_played,
                        (current_pace_per_40 * (games_played - 1) + row['PACE/40']) / games_played,
                        (current_pie * (games_played - 1) + row['PIE']) / games_played,
                        (current_percentagefga_twopt * (games_played - 1) + row['%FGA 2PT']) / games_played,
                        (current_percentagefga_threept * (games_played - 1) + row['%FGA 3PT']) / games_played,
                        (current_percentagepts_twopt * (games_played - 1) + row['%PTS 2PT']) / games_played,
                        (current_percentagepts_twopt_mr * (games_played - 1) + row['%PTS 2PT MR']) / games_played,
                        (current_percentagepts_threept * (games_played - 1) + row['%PTS 3PT']) / games_played,
                        (current_percentagepts_fbps * (games_played - 1) + row['%PTS FBPs']) / games_played,
                        (current_percentagepts_ft * (games_played - 1) + row['%PTS FT']) / games_played,
                        (current_percentagepts_offto * (games_played - 1) + row['%PTS OffTO']) / games_played,
                        (current_percentagepts_pitp * (games_played - 1) + row['%PTS PITP']) / games_played,
                        (current_twofgm_percentageast * (games_played - 1) + row['2FGM %AST']) / games_played,
                        (current_twofgm_percentageuast * (games_played - 1) + row['2FGM %UAST']) / games_played,
                        (current_threefgm_percentageast * (games_played - 1) + row['3FGM %AST']) / games_played,
                        (current_threefgm_percentageuast * (games_played - 1) + row['3FGM %UAST']) / games_played,
                        (current_fgm_percentageast * (games_played - 1) + row['FGM %AST']) / games_played,
                        (current_fgm_percentageuast * (games_played - 1) + row['FGM %UAST']) / games_played,
                        (current_usgpercentage * (games_played - 1) + row['USG%']) / games_played,
                        (current_rebpercentage * (games_played - 1) + row['REB%']) / games_played,
                        games_played
                    ]

                    # Update the table with the new averages
                    update_query = sql.SQL("""
                        UPDATE stat_avg_season
                        SET fga = %s, fgm = %s, threepa = %s, threepm = %s, fta = %s, ftm = %s, pts = %s, reb = %s, oreb = %s, dreb = %s, 
                            ast = %s, tov = %s, pf = %s, stl = %s, blk = %s, pace = %s, pace_per_40 = %s, pie = %s, percentagefga_twopt = %s, 
                            percentagefga_threept = %s, percentagepts_twopt = %s, percentagepts_twopt_mr = %s, percentagepts_threept = %s, 
                            percentagepts_fbps = %s, percentagepts_ft = %s, percentagepts_offto = %s, percentagepts_pitp = %s, 
                            twofgm_percentageast = %s, twofgm_percentageuast = %s, threefgm_percentageast = %s, threefgm_percentageuast = %s, 
                            fgm_percentageast = %s, fgm_percentageuast = %s, usgpercentage = %s, rebpercentage = %s, games_played = %s
                        WHERE player = %s
                    """)

                    cur.execute(update_query, updated_row + [player])
                    
                    player_name = row['PLAYER']
                    opp = row['opp']

                    # Step 2: Find the minimum games_since for the player, if any
                    cur.execute(
                        sql.SQL("SELECT MIN(games_since) FROM stat_lag WHERE player = %s"),
                        [player_name]
                    )
                    
                    min_games_since = cur.fetchone()[0]- 1


                    cur.execute(
                        sql.SQL("DELETE FROM stat_lag WHERE player = %s AND games_since = %s"),
                        [player_name, min_games_since+5]
                    )

                    # Step 4: Insert the new row into the table
                    insert_query = sql.SQL("""
        INSERT INTO stat_lag (
            player, games_since, fga, fgm, threepa, threepm, fta, ftm, pts, pf, reb, oreb, dreb, ast, tov, stl, blk, pace, pace_per_40, 
            pie, percentagefga_twopt, percentagefga_threept, percentagepts_twopt, percentagepts_twopt_mr, percentagepts_threept, 
            percentagepts_fbps, percentagepts_ft, percentagepts_offto, percentagepts_pitp, twofgm_percentageast, 
            twofgm_percentageuast, threefgm_percentageast, threefgm_percentageuast, fgm_percentageast, fgm_percentageuast, usgpercentage, 
                                           rebpercentage
        ) VALUES (%s, %s, %s, %s, %s, %s, 
                  %s, %s, %s, %s, %s, %s, 
                  %s, %s, %s, %s, %s, %s, 
                  %s, %s, %s, %s, %s, %s, 
                  %s, %s, %s, %s, %s, %s, 
                  %s, %s, %s, %s, %s, %s,%s)
    """)

                    cur.execute(insert_query, [
                        row['PLAYER'], min_games_since, row['FGA'], row['FGM'], row['3PA'], row['3PM'], 
                        row['FTA'], row['FTM'], row['PTS'], row['PF'], row['REB'], row['OREB'], 
                        row['DREB'], row['AST'], row['TOV'], row['STL'], row['BLK'], row['PACE'], 
                        row['PACE/40'], row['PIE'], row['%FGA 2PT'], row['%FGA 3PT'], row['%PTS 2PT'], row['%PTS 2PT MR'], 
                        row['%PTS 3PT'], row['%PTS FBPs'], row['%PTS FT'], row['%PTS OffTO'], row['%PTS PITP'], row['2FGM %AST'], 
                        row['2FGM %UAST'], row['3FGM %AST'], row['3FGM %UAST'], row['FGM %AST'], row['FGM %UAST'], row['USG%'], 
                        row['REB%']
                    ])

                    

                    #STAT_LAG_OPP
                    #select * from stat_lag_opp where player = 'Caitlin Clark' and opp = 'dal' and games_since = (SELECT MAX(games_since) FROM stat_lag_opp WHERE player = 'Caitlin Clark' and opp = 'dal');


                    # Step 2: Find the minimum games_since for the player, if any

                    cur.execute(
                        sql.SQL("DELETE FROM stat_lag_opp WHERE player = %s AND opp = %s"),
                        [player_name, opp]
                    )



                    # Step 4: Insert the new row into the table
                    insert_query = sql.SQL("""
                        INSERT INTO stat_lag_opp (
                            player, opp, fga, fgm, threepa, threepm, fta, ftm, pts, pf, reb, oreb, dreb, ast, tov, stl, blk, pace, pace_per_40, pie, percentagefga_twopt, percentagefga_threept, percentagepts_twopt, percentagepts_twopt_mr, percentagepts_threept, percentagepts_fbps, percentagepts_ft, percentagepts_offto, percentagepts_pitp, twofgm_percentageast, twofgm_percentageuast, threefgm_percentageast, threefgm_percentageuast, fgm_percentageast, fgm_percentageuast, usgpercentage, rebpercentage
                        ) VALUES (%s, %s, %s, %s, %s, %s, 
                                  %s, %s, %s, %s, %s, %s, 
                                  %s, %s, %s, %s, %s, %s, 
                                  %s, %s, %s, %s, %s, %s, 
                                  %s, %s, %s, %s, %s, %s, 
                                  %s, %s, %s, %s, %s, %s,%s)
                    """)

                    cur.execute(insert_query, [
                        row['PLAYER'], opp, row['FGA'], row['FGM'], row['3PA'], row['3PM'], 
                        row['FTA'], row['FTM'], row['PTS'], row['PF'], row['REB'], row['OREB'], 
                        row['DREB'], row['AST'], row['TOV'], row['STL'], row['BLK'], row['PACE'], 
                        row['PACE/40'], row['PIE'], row['%FGA 2PT'], row['%FGA 3PT'], row['%PTS 2PT'], row['%PTS 2PT MR'], 
                        row['%PTS 3PT'], row['%PTS FBPs'], row['%PTS FT'], row['%PTS OffTO'], row['%PTS PITP'], row['2FGM %AST'], 
                        row['2FGM %UAST'], row['3FGM %AST'], row['3FGM %UAST'], row['FGM %AST'], row['FGM %UAST'], row['USG%'], 
                        row['REB%']
                    ])
            for index,row in team_df.iterrows():
            # Step 1: Find the minimum games_since for the team, if any
            
                team = row['TEAM']
                opp = row['OPP']
            
            # Retrieve the current values from the table
                cur.execute(
                sql.SQL("""
                    SELECT games_played, pts, reb, oreb, dreb, ast, pf, stl, tov, blk, fga, fgm, threepa, threepm, fta, ftm,
                        offrtg, defrtg, netrtg, astpercentage, ast_per_to, ast_ratio, orebpercentage, drebpercentage, rebpercentage,
                        tovpercentage, efgpercentage, tspercentage, pace, pace_per_40, pie
                    FROM team_avg 
                    WHERE team = %s
                """),
                [team]
            )

                result = cur.fetchone()

                # If the team exists in the table, update the averages
                if result:
                    (
                        games_played, current_pts, current_reb, current_oreb, current_dreb, current_ast, current_pf, current_stl, 
                        current_tov, current_bs, current_fga, current_fgm, current_threepa, current_threepm, current_fta, current_ftm,
                        current_offrtg, current_defrtg, current_netrtg, current_astpercentage, current_ast_per_to, current_ast_ratio,
                        current_orebpercentage, current_drebpercentage, current_rebpercentage, current_tovpercentage, current_efgpercentage,
                        current_tspercentage, current_pace, current_pace_per_40, current_pie
                    ) = result

                    # Update games played and calculate new averages
                    games_played += 1
                    updated_row = [
                        (current_pts * (games_played - 1) + row['PTS']) / games_played,
                        (current_reb * (games_played - 1) + row['REB']) / games_played,
                        (current_oreb * (games_played - 1) + row['OREB']) / games_played,
                        (current_dreb * (games_played - 1) + row['DREB']) / games_played,
                        (current_ast * (games_played - 1) + row['AST']) / games_played,
                        (current_pf * (games_played - 1) + row['PF']) / games_played,
                        (current_stl * (games_played - 1) + row['STL']) / games_played,
                        (current_tov * (games_played - 1) + row['TOV']) / games_played,
                        (current_bs * (games_played - 1) + row['BLK']) / games_played,
                        (current_fga * (games_played - 1) + row['FGA']) / games_played,
                        (current_fgm * (games_played - 1) + row['FGM']) / games_played,
                        (current_threepa * (games_played - 1) + row['3PA']) / games_played,
                        (current_threepm * (games_played - 1) + row['3PM']) / games_played,
                        (current_fta * (games_played - 1) + row['FTA']) / games_played,
                        (current_ftm * (games_played - 1) + row['FTM']) / games_played,
                        # Update all other stats similarly
                        (current_offrtg * (games_played - 1) + row['OFFRTG']) / games_played,
                        (current_defrtg * (games_played - 1) + row['DEFRTG']) / games_played,
                        (current_netrtg * (games_played - 1) + row['NETRTG']) / games_played,
                        (current_astpercentage * (games_played - 1) + row['AST%']) / games_played,
                        (current_ast_per_to * (games_played - 1) + row['AST/TO']) / games_played,
                        (current_ast_ratio * (games_played - 1) + row['AST RATIO']) / games_played,
                        (current_orebpercentage * (games_played - 1) + row['OREB%']) / games_played,
                        (current_drebpercentage * (games_played - 1) + row['DREB%']) / games_played,
                        (current_rebpercentage * (games_played - 1) + row['REB%']) / games_played,
                        (current_tovpercentage * (games_played - 1) + row['TOV%']) / games_played,
                        (current_efgpercentage * (games_played - 1) + row['EFG%']) / games_played,
                        (current_tspercentage * (games_played - 1) + row['TS%']) / games_played,
                        (current_pace * (games_played - 1) + row['PACE']) / games_played,
                        (current_pace_per_40 * (games_played - 1) + row['PACE/40']) / games_played,
                        (current_pie * (games_played - 1) + row['PIE']) / games_played
                    ]

                    # Update the table with the new averages
                    update_query = sql.SQL("""
                        UPDATE team_avg
                        SET games_played = %s, pts = %s, reb = %s, oreb = %s, dreb = %s, ast = %s, pf = %s, stl = %s, tov = %s, blk = %s,
                            fga = %s, fgm = %s, threepa = %s, threepm = %s, fta = %s, ftm = %s,
                            offrtg = %s, defrtg = %s, netrtg = %s, astpercentage = %s, ast_per_to = %s, ast_ratio = %s, orebpercentage = %s, 
                            drebpercentage = %s, rebpercentage = %s, tovpercentage = %s, efgpercentage = %s, tspercentage = %s, 
                            pace = %s, pace_per_40 = %s, pie = %s
                        WHERE team = %s
                    """)

                    cur.execute(update_query, [games_played] + updated_row + [team])




                cur.execute(
                    sql.SQL("""
                        SELECT games_played, pts, reb, oreb, dreb, ast, pf, stl, tov, blk, fga, fgm, threepa, threepm, fta, ftm,
                            offrtg, defrtg, netrtg, astpercentage, ast_per_to, ast_ratio, orebpercentage, drebpercentage, rebpercentage,
                            tovpercentage, efgpercentage, tspercentage, pace, pace_per_40, pie
                        FROM team_allowed_avg 
                        WHERE team = %s
                    """),
                    [opp]
                )

                result = cur.fetchone()

                # If the team exists in the table, update the averages
                if result:
                    (
                        games_played, current_pts, current_reb, current_oreb, current_dreb, current_ast, current_pf, current_stl, 
                        current_tov, current_bs, current_fga, current_fgm, current_threepa, current_threepm, current_fta, current_ftm,
                        current_offrtg, current_defrtg, current_netrtg, current_astpercentage, current_ast_per_to, current_ast_ratio,
                        current_orebpercentage, current_drebpercentage, current_rebpercentage, current_tovpercentage, current_efgpercentage,
                        current_tspercentage, current_pace, current_pace_per_40, current_pie
                    ) = result

                    # Update games played and calculate new averages
                    games_played += 1
                    updated_row = [
                        (current_pts * (games_played - 1) + row['PTS']) / games_played,
                        (current_reb * (games_played - 1) + row['REB']) / games_played,
                        (current_oreb * (games_played - 1) + row['OREB']) / games_played,
                        (current_dreb * (games_played - 1) + row['DREB']) / games_played,
                        (current_ast * (games_played - 1) + row['AST']) / games_played,
                        (current_pf * (games_played - 1) + row['PF']) / games_played,
                        (current_stl * (games_played - 1) + row['STL']) / games_played,
                        (current_tov * (games_played - 1) + row['TOV']) / games_played,
                        (current_bs * (games_played - 1) + row['BLK']) / games_played,
                        (current_fga * (games_played - 1) + row['FGA']) / games_played,
                        (current_fgm * (games_played - 1) + row['FGM']) / games_played,
                        (current_threepa * (games_played - 1) + row['3PA']) / games_played,
                        (current_threepm * (games_played - 1) + row['3PM']) / games_played,
                        (current_fta * (games_played - 1) + row['FTA']) / games_played,
                        (current_ftm * (games_played - 1) + row['FTM']) / games_played,
                        # Update all other stats similarly
                        (current_offrtg * (games_played - 1) + row['OFFRTG']) / games_played,
                        (current_defrtg * (games_played - 1) + row['DEFRTG']) / games_played,
                        (current_netrtg * (games_played - 1) + row['NETRTG']) / games_played,
                        (current_astpercentage * (games_played - 1) + row['AST%']) / games_played,
                        (current_ast_per_to * (games_played - 1) + row['AST/TO']) / games_played,
                        (current_ast_ratio * (games_played - 1) + row['AST RATIO']) / games_played,
                        (current_orebpercentage * (games_played - 1) + row['OREB%']) / games_played,
                        (current_drebpercentage * (games_played - 1) + row['DREB%']) / games_played,
                        (current_rebpercentage * (games_played - 1) + row['REB%']) / games_played,
                        (current_tovpercentage * (games_played - 1) + row['TOV%']) / games_played,
                        (current_efgpercentage * (games_played - 1) + row['EFG%']) / games_played,
                        (current_tspercentage * (games_played - 1) + row['TS%']) / games_played,
                        (current_pace * (games_played - 1) + row['PACE']) / games_played,
                        (current_pace_per_40 * (games_played - 1) + row['PACE/40']) / games_played,
                        (current_pie * (games_played - 1) + row['PIE']) / games_played
                    ]

                    # Update the table with the new averages
                    update_query = sql.SQL("""
                        UPDATE team_allowed_avg
                        SET games_played = %s, pts = %s, reb = %s, oreb = %s, dreb = %s, ast = %s, pf = %s, stl = %s, tov = %s, blk = %s,
                            fga = %s, fgm = %s, threepa = %s, threepm = %s, fta = %s, ftm = %s,
                            offrtg = %s, defrtg = %s, netrtg = %s, astpercentage = %s, ast_per_to = %s, ast_ratio = %s, orebpercentage = %s, 
                            drebpercentage = %s, rebpercentage = %s, tovpercentage = %s, efgpercentage = %s, tspercentage = %s, 
                            pace = %s, pace_per_40 = %s, pie = %s
                        WHERE team = %s
                    """)

                    cur.execute(update_query, [games_played] + updated_row + [opp])
                


                cur.execute(
                    sql.SQL("SELECT MIN(games_since) FROM team_for_lag WHERE team = %s"),
                    [row['TEAM']]
                )
                min_games_since = cur.fetchone()[0] -1

                if min_games_since is not None:
                    # Step 2: Delete a specific record if needed (adjust logic as per requirements)
            
    # Step 2: Delete a specific record if needed (adjust logic as per requirements)
                    cur.execute(
                        sql.SQL("DELETE FROM team_for_lag WHERE team = %s AND games_since = %s"),
                        [row['TEAM'], min_games_since + 5]
                    )

                # Step 3: Insert the new row into the table
                insert_query = sql.SQL("""
                    INSERT INTO team_for_lag (
                        team, games_since, offrtg, defrtg, netrtg, astpercentage, 
                        ast_per_to, ast_ratio, orebpercentage, drebpercentage, rebpercentage, tovpercentage, 
                        efgpercentage,tspercentage, pace, pace_per_40, pie, pts, 
                        reb, oreb, dreb, ast, pf, stl, 
                        tov, blk, fga, fgm, threepa, threepm, 
                                       fta, ftm
                    ) VALUES (
                        %s, %s, %s, %s, %s, %s, 
                        %s, %s, %s, %s, %s, %s, 
                        %s, %s, %s, %s, %s, %s,
                        %s, %s, %s, %s, %s, %s,
                        %s, %s, %s, %s, %s, %s, %s, %s
                    )
                    ON CONFLICT (team, games_since)
                    DO NOTHING
                """)

                cur.execute(insert_query, (
                    row['TEAM'], min_games_since, row['OFFRTG'], row['DEFRTG'], row['NETRTG'], row['AST%'], 
                    row['AST/TO'], row['AST RATIO'], row['OREB%'], row['DREB%'], row['REB%'], row['TOV%'], 
                    row['EFG%'], row['TS%'], row['PACE'], row['PACE/40'], row['PIE'], row['PTS'], 
                    row['REB'], row['OREB'], row['DREB'], row['AST'], row['PF'], row['STL'], 
                    row['TOV'], row['BLK'], row['FGA'], row['FGM'], row['3PA'], row['3PM'], 
                    row['FTA'], row['FTM']
                ))

                cur.execute(
                sql.SQL("SELECT MIN(games_since) FROM team_allowed_lag WHERE team = %s"),
                [row['OPP']]
            )
                min_games_since = cur.fetchone()[0] -1 

                if min_games_since is not None:
                    # Step 2: Delete a specific record if needed (adjust logic as per requirements)
                    cur.execute(
                        sql.SQL("DELETE FROM team_allowed_lag WHERE team = %s AND games_since = %s"),
                        [row['OPP'], min_games_since + 5]
                    )
                conn.commit()

                # Step 3: Insert the new row into the table
                insert_query = sql.SQL("""
                    INSERT INTO team_allowed_lag (
                        team, games_since, offrtg, defrtg, netrtg, astpercentage, ast_per_to, ast_ratio,
                        orebpercentage, drebpercentage, rebpercentage, tovpercentage, efgpercentage,
                        tspercentage, pace, pace_per_40, pie, pts, reb, oreb, dreb, ast, pf, stl, tov,
                        blk, fga, fgm, threepa, threepm, fta, ftm
                    ) VALUES (
                        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                                       %s,%s
                    )
                    ON CONFLICT (team, games_since)
                    DO NOTHING
                """)


                cur.execute(insert_query, (
                    row['TEAM'], min_games_since, row['OFFRTG'], row['DEFRTG'], row['NETRTG'], row['AST%'], 
                    row['AST/TO'], row['AST RATIO'], row['OREB%'], row['DREB%'], 
                    row['REB%'], row['TOV%'], row['EFG%'], row['TS%'], 
                    row['PACE'], row['PACE/40'], row['PIE'], row['PTS'], row['REB'], row['OREB'], 
                    row['DREB'], row['AST'], row['PF'], row['STL'], row['TOV'], row['BLK'], row['FGA'], 
                    row['FGM'], row['3PA'], row['3PM'], row['FTA'], row['FTM']

                ))
                cur.execute(
                sql.SQL("SELECT MIN(games_since) FROM matchup WHERE team = %s AND opp = %s"),
                [row['TEAM'], row['OPP']]
                )
                min_games_since = cur.fetchone()[0] -1 

                if min_games_since is not None:
                    # Step 2: Delete a specific record if needed (adjust logic as per requirements)
                    cur.execute(
                        sql.SQL("DELETE FROM matchup WHERE team = %s AND opp = %s AND games_since = %s"),
                        [row['TEAM'], row['OPP'], min_games_since + 4]
                    )
                # Step 3: Insert the new row into the table
                insert_query = sql.SQL("""
                    INSERT INTO matchup (
                        team, opp, games_since, fga, fgm, threepa, threepm, fta, ftm, 
                        pts, reb, oreb, dreb, ast, tov, pf, stl, blk, pace_per_40, pace, pie
                    ) VALUES (
                        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
                    )
                    ON CONFLICT (team, opp, games_since)
                    DO NOTHING
                """)

                cur.execute(insert_query, (
                    row['TEAM'], row['OPP'], min_games_since, row['FGA'], row['FGM'], row['3PA'], 
                    row['3PM'], row['FTA'], row['FTM'], row['PTS'], row['REB'], row['OREB'], 
                    row['DREB'], row['AST'], row['TOV'], row['PF'], row['STL'], row['BLK'], 
                    row['PACE/40'], row['PACE'], row['PIE']
                ))


            for index,row in pos_df.iterrows():
                team = row['Team']
                pos = row['pos']
                cur.execute(
                sql.SQL("""
                    SELECT games_played, pts, reb, oreb, dreb, fga, ast
                    FROM pos_allowed
                    WHERE team = %s AND pos = %s
                """),
                [team, pos]
            )

                result = cur.fetchone()

                # If the team and pos exist in the table, update the averages
                if result:
                    (
                        games_played, current_pts, current_reb, current_oreb, current_dreb, current_fga, current_ast
                    ) = result

                    # Update games played and calculate new averages
                    games_played += 1
                    updated_row = [
                        (current_pts * (games_played - 1) + row['PTS']) / games_played,
                        (current_reb * (games_played - 1) + row['REB']) / games_played,
                        (current_oreb * (games_played - 1) + row['OREB']) / games_played,
                        (current_dreb * (games_played - 1) + row['DREB']) / games_played,
                        (current_fga * (games_played - 1) + row['FGA']) / games_played,
                        (current_ast * (games_played - 1) + row['AST']) / games_played
                    ]
                    
                    # Prepare the update query
                    cur.execute(
                        sql.SQL("""
                            UPDATE pos_allowed
                            SET games_played = %s, pts = %s, reb = %s, oreb = %s, dreb = %s, fga = %s, ast = %s
                            WHERE team = %s AND pos = %s
                        """),   
                        [
                            games_played, updated_row[0], updated_row[1], updated_row[2],
                            updated_row[3], updated_row[4], updated_row[5], team, pos
                        ]
                    )
                    cur.execute(
                    sql.SQL("SELECT MIN(games_since) FROM pos_allowed_lag WHERE team = %s AND pos = %s"),
                    [row['Team'], row['pos']]
                )
                    min_games_since = cur.fetchone()[0] - 1

                    if min_games_since is not None:
                        # Step 2: Delete a specific record if needed (adjust logic as per requirements)
                        cur.execute(
                            sql.SQL("DELETE FROM pos_allowed_lag WHERE team = %s AND pos = %s AND games_since = %s"),
                            [row['Team'], row['pos'], min_games_since + 5]
                        )

                    # Step 3: Insert the new row into the table
                    insert_query = sql.SQL("""
                        INSERT INTO pos_allowed_lag (
                            team, games_since, pos, pts, reb, oreb, dreb, fga, ast
                        ) VALUES (
                            %s, %s, %s, %s, %s, %s, %s, %s, %s
                        )
                        ON CONFLICT (team, pos, games_since)
                        DO NOTHING
                    """)

                    cur.execute(insert_query, (
                        row['opp'], min_games_since, row['pos'], row['PTS'], row['REB'], row['OREB'], 
                        row['DREB'], row['FGA'], row['AST']
                    ))

            conn.commit()
        

        if close:
            cur.close()
            conn.close()

In [201]:
rebinsert_stats(conn,player_df,team_stat_df, pos_team_df)

777    21.0
784     2.0
783     5.0
781    10.0
780     3.0
       ... 
34      0.0
33      2.0
32      7.0
30      0.0
0       2.0
Name: PTS, Length: 806, dtype: float64


In [44]:
import database

db=database.Database()


In [1]:
import pandas as pd
import psycopg2
from psycopg2 import sql
import warnings
import pandas as pd

# Suppress the specific warning
warnings.filterwarnings("ignore", category=UserWarning, message="pandas only supports SQLAlchemy")
import json
from bs4 import BeautifulSoup
import regex as re
from tabulate import tabulate
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
import time
import numpy as np
from tensorflow.keras.models import load_model

In [11]:
import pickle
import database

In [12]:
db = database.Database()

In [28]:
def generate_predictions():
        home_set = set(("WAS",'NYL','IND','CHI','LAS'))
        models = []
        stat_dict = {'Offensive Rebounds':0,'Defensive Rebounds':1, 'Points':2 ,'Pts+Rebs+Asts':3, 'FG Attempted':4,
                    'Rebounds': 5,'Assists':6 }
        for model in ['oreb','dreb','pts','pra','fga','reb','ast']:
                loaded_model = load_model(f'nn_{model}_4.keras')
                with open(f'xgb_{model}.pkl', 'rb') as f:
                        pipeline_loaded = pickle.load(f)
                models.append((loaded_model,pipeline_loaded))
        # Load the XGBoost model
        db = database.Database()
        with open('prizepick_lines.txt', 'r') as file:
                text = file.read()
                lines = db.get_teams(text)
        prediction_results = []
        for _,row in lines.iterrows():
                try:
                        if row['attributes.stat_type'] not in stat_dict:
                                continue
                        player = row['player'].replace("'",'').lower().title()
                        team = row['team']
                        opp = row['opp']
                        nn,xgb = models[stat_dict[row['attributes.stat_type']]]
                        if team in home_set:
                                home = 1
                        else:
                                home = 0
                        un_aligned = db.get_row(player,team,opp,home, ['pts','reb','ast','oreb','dreb','fga'],season=28, close = False)
                        aligned = db.realign_df(un_aligned)
        
                        prediction_nn = nn.predict(aligned)
                        prediction_xgb = xgb.predict(aligned)
                        prediction_val = .4*prediction_nn + .6*prediction_xgb
                        if prediction_val < float(row['attributes.line_score']):
                                Over_Under = 'under'
                        else:
                                Over_Under = 'over'
                        prediction = {'name': player,
                        'team': team,
                        'prediction': round(float(prediction_val[0]),2),
                        'line': float(row['attributes.line_score']),
                        'stat': row['attributes.stat_type'],
                        'odds': 'none',
                        'book': 'pp',
                        'OU': Over_Under}
                        
                        prediction_results.append(prediction)
                except Exception as e:
                        print(e)

        return prediction_results

In [29]:
generate_predictions()

c:\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step


C:\Users\00011\AppData\Local\Temp\ipykernel_18404\3066495278.py:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  'prediction': round(float(prediction_val[0]),2),


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step


C:\Users\00011\AppData\Local\Temp\ipykernel_18404\3066495278.py:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  'prediction': round(float(prediction_val[0]),2),


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


C:\Users\00011\AppData\Local\Temp\ipykernel_18404\3066495278.py:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  'prediction': round(float(prediction_val[0]),2),
C:\Users\00011\AppData\Local\Temp\ipykernel_18404\3066495278.py:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  'prediction': round(float(prediction_val[0]),2),


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


C:\Users\00011\AppData\Local\Temp\ipykernel_18404\3066495278.py:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  'prediction': round(float(prediction_val[0]),2),


[{'name': 'Caitlin Clark',
  'team': 'IND',
  'prediction': 0.35,
  'line': 0.5,
  'stat': 'Offensive Rebounds',
  'odds': 'none',
  'book': 'pp',
  'OU': 'under'},
 {'name': 'Caitlin Clark',
  'team': 'IND',
  'prediction': 9.71,
  'line': 9.5,
  'stat': 'Assists',
  'odds': 'none',
  'book': 'pp',
  'OU': 'over'},
 {'name': 'Caitlin Clark',
  'team': 'IND',
  'prediction': 32.7,
  'line': 38.5,
  'stat': 'Pts+Rebs+Asts',
  'odds': 'none',
  'book': 'pp',
  'OU': 'under'},
 {'name': 'Caitlin Clark',
  'team': 'IND',
  'prediction': 20.75,
  'line': 23.0,
  'stat': 'Points',
  'odds': 'none',
  'book': 'pp',
  'OU': 'under'},
 {'name': 'Caitlin Clark',
  'team': 'IND',
  'prediction': 16.19,
  'line': 17.0,
  'stat': 'FG Attempted',
  'odds': 'none',
  'book': 'pp',
  'OU': 'under'},
 {'name': 'Kelsey Mitchell',
  'team': 'IND',
  'prediction': 15.87,
  'line': 16.5,
  'stat': 'FG Attempted',
  'odds': 'none',
  'book': 'pp',
  'OU': 'under'},
 {'name': 'Kelsey Mitchell',
  'team': 'IN

In [15]:
with open('prizepick_lines.txt', 'r') as file:
                text = file.read()
                lines = db.get_teams(text)

In [9]:
lines['attributes.stat_type'].unique()

NameError: name 'lines' is not defined

In [3]:
lines

,relationships.new_player.data.id,attributes.stat_type,attributes.line_score,attributes.description
0,A'ja Wilson + Caitlin Clark,Points (Combo),50.5,IND/LVA
1,A'ja Wilson + Aliyah Boston,Rebounds (Combo),21.0,IND/LVA
2,Chelsea Gray + Caitlin Clark,Assists (Combo),14.5,IND/LVA
3,Kelsey Plum + Caitlin Clark,3-PT Made (Combo),6.5,IND/LVA
4,Jackie Young + Kelsey Mitchell,3-PT Made (Combo),5.0,IND/LVA
...,...,...,...,...
191,Ezi Magbegor,Pts+Rebs,17.5,LAS
192,Ezi Magbegor,Rebounds,7.0,LAS
193,Ezi Magbegor,Fantasy Score,28.0,LAS
194,Ezi Magbegor,Pts+Rebs+Asts,20.0,LAS
